Imports

In [1]:
import ray
import os
os.environ["OPENCV_LOG_LEVEL"] = "SILENT"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
for devices in physical_devices:
    tf.config.experimental.set_memory_growth(devices, True)

from sklearn.metrics import classification_report
import tensorflow_model_optimization as tfmot
import tflite_runtime.interpreter as tflite
from dvclive import Live
import numpy as np
import keras
import cv2

2023-12-22 11:00:09.955468: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 11:00:09.955515: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 11:00:09.955541: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Quantization

In [ ]:
keras_model = tf.keras.models.load_model('Models/768Efficientnetv2B0.keras')
quantize_config = tfmot.quantization.keras.QuantizeConfig(weight_dtype=tf.float16)
annotated_model = tfmot.quantization.keras.quantize_annotate_model(
    keras_model, quantize_config=quantize_config)
q_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)


Loading Dataset

In [ ]:
dim =768
image_size = (dim, dim)
batch_size = 20

train_ds, other = keras.utils.image_dataset_from_directory(
    "dataset/dataset_checkin",
    validation_split=0.26,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

bch = other.cardinality().numpy()
bch /= 2
test_ds, val_ds = other.take(bch), other.skip(bch).take(bch)


y_test = np.concatenate([y.numpy() for _, y in test_ds])
label = {0:'left', 1:'lower', 2:'right', 3:'smile', 4:'upper'}
y_test = [label[i] for i in y_test]

X_test = np.expand_dims(np.concatenate([x.numpy() for x, _ in test_ds]), axis=1)

In [ ]:
y_pred = q_aware_model.predict(test_ds)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import tensorflow as tf
from keras.applications import EfficientNetV2B0
import tensorflow_model_optimization as tfmot

# Load the EfficientNetV2B0 model from tf.keras.applications
efficientnetv2b0_model = EfficientNetV2B0(weights='imagenet', include_top=True)

# Clone the model to avoid modifying the original one
keras_model = tf.keras.models.clone_model(efficientnetv2b0_model)

# Define a function to create a quantization config for each layer
def apply_quantization(layer):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)

# Annotate the entire model with the quantization config
annotated_model = tf.keras.models.clone_model(
    keras_model,
    clone_function=apply_quantization
)

# Choose the quantization scheme
quantize_scheme = tfmot.quantization.keras.QuantizeConfig(
    weight_quantizer=tfmot.quantization.keras.quantizers.Quantizer,
    activation_quantizer=tfmot.quantization.keras.quantizers.Quantizer
)

# Apply the quantization scheme to the annotated model
quantized_model = tfmot.quantization.keras.quantize_apply(
    annotated_model,
    quantize_config=quantize_scheme
)

# Compile the quantized model with the same optimizer, loss, and metrics
# quantized_model.compile(optimizer='adam',
                        # loss='sparse_categorical_crossentropy',
                        # metrics=['accuracy'])

# Fine-tune the quantized model on your dataset if needed
# quantized_model.fit(...)

# Save the quantized model
# quantized_model.save('quantized_efficientnetv2b0.h5')
